<a href="https://colab.research.google.com/github/zzbatmos/Polarization_Book/blob/main/Compiling_Mie_PDA_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Try to compile the Mie code and Polarizaed Adding Doubling in Colab

install gfortran compiler and netCDF library needed to compile the fortran code

In [ ]:
!apt-get install gfortran g++ gcc libnetcdf-dev libnetcdff-dev &> /dev/null

Mount Google Drive

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Compile and run the Mie code

Check the netCDF configurations
use the information with "--libs" in the Makefile of Mie code

In [ ]:
!nc-config --all


This netCDF 4.6.0 has been built with the following features: 

  --cc        -> /usr/bin/cc
  --cflags    -> -I/usr/include -I/usr/include/hdf5/serial
  --libs      -> -L/usr/lib/x86_64-linux-gnu -L/usr/lib/x86_64-linux-gnu/hdf5/serial -lnetcdf -lhdf5_hl -lhdf5 -lpthread -lsz -lz -ldl -lm -lcurl

  --has-c++   -> no
  --cxx       -> 

  --has-c++4  -> no
  --cxx4      -> 

  --has-fortran-> yes
  --fc        -> gfortran
  --fflags    -> -I/usr/include
  --flibs     -> -L/usr/lib -lnetcdff -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-z,now -lnetcdf -lnetcdf
  --has-f90   -> no
  --has-f03   -> yes

  --has-dap   -> yes
  --has-dap2  -> yes
  --has-dap4  -> yes
  --has-nc2   -> yes
  --has-nc4   -> yes
  --has-hdf5  -> yes
  --has-hdf4  -> no
  --has-logging-> no
  --has-pnetcdf-> no
  --has-szlib -> no
  --has-cdf5 -> no
  --has-parallel-> no

  --prefix    -> /usr
  --includedir-> /usr/include
  --libdir    -> /usr/lib/x86_64-linux-gnu
  --version   -> netCDF 4.6.0



 Go the folder of Mie Code and Compile it

In [ ]:
%cd /content/drive/Shareddrives/Polarization_Book/Code/Mie_code
!make 

/content/drive/Shareddrives/Polarization_Book/Code/Mie_code
gfortran -O2 -g -o mie_single_size.exe mie_single_size.f90 MIEV0.o RDI1MACHmodule.o  ErrPack.o write_data.o -I/usr/include -L/usr/lib -lnetcdff -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-z,now -lnetcdf -lnetcdf
gfortran -O2 -g -o mie_single_size_driver.exe driver.f90 gamma_size.o REFWAT.o


After compiling, run a test case. This will generate a new netCDF file "Mie_Code_output.nc"

In [ ]:
import subprocess
import sys
result = subprocess.run( "./mie_single_size.exe",capture_output=True, text=True)

In [ ]:
%ls -lt *.nc

-rw------- 1 root root 86503940 Dec 17 01:23 Mie_Code_output.nc
-rw------- 1 root root 86503940 Dec 16 17:49 Single_Scattering_Mie_CloudSat.nc
-rw------- 1 root root    36772 Nov 18  2013 test.nc
-rw------- 1 root root   296152 Sep 30  2011 POLDER_443p_Re1to50.nc


## Compiling the Polarizaed Doubling Adding (PDA)

In [81]:
%cd /content/drive/Shareddrives/Polarization_Book/Code/Adding_Doubling/rt_code/
%cd lib/
%rm *.o *.a
!gfortran -m64 -frecord-marker=4 -c  `ls *.f`
!gfortran -m64 -frecord-marker=4 -c  `ls *.f90`
!ar -r libvec.a *.o
!ranlib libvec.a
%cd ..


/content/drive/Shareddrives/Polarization_Book/Code/Adding_Doubling/rt_code
/content/drive/Shareddrives/Polarization_Book/Code/Adding_Doubling/rt_code/lib
ar: creating libvec.a
/content/drive/Shareddrives/Polarization_Book/Code/Adding_Doubling/rt_code


In [82]:
!gfortran -m64 -frecord-marker=4 -o vec_generate_ops vec_generate_ops.f -Llib -lvec
!gfortran -m64 -frecord-marker=4 -o vec_interp vec_interp.f -Llib -lvec
!gfortran -m64 -frecord-marker=4 -o vec_srf vec_srf.f -Llib -lvec

In [87]:
!./vec_generate_ops ./ Rayleigh_0.46.info ./

 /content/drive/Shareddrives/Polarization_Book/Code/Adding_Doubling/rt_code/../ocean/OC_Fmatrix_all.txt                                                              
 ./                                                                              Rayleigh_0.46.info                                                              ./                                                                              
          1          1          1          0          1          1
Number of aerosol types, wavelengths, vertical layers and flag for absorption   
 MCAP= 90 NCAP= 18 NCAP2= 18NTAU= 24 NTAU2= 24 MTOT= 15 N3BY3= 7
 QSTOP= 0.10D-19  QSTOP2= 0.10D-19
 NTHETA, THE NUMBER OF GAUSS POINTS= 56
 NEXTRA=  1
 SUM OF GAUSS WEIGHTS=    1.00000000000000 SUM OF WM'S=    1.00000000000000
NPHI THE NUMBER OF GAUSS POINTS FOR PHI INTEGRATION =  25
 SUM OF PHI WEIGHTS=    1.00000000000000 SUM OF PHI=    1.00000000000000
 ERRBNDR= 0.20D-03  ERRBNDP= 0.10D-03
 NPERS =  2
  90.00000   0.86600
Definitions of

In [90]:
!./vec_interp ./ Rayleigh_0.46 ./ 30 60 5 0 180 20 1

 ./                                                                              Rayleigh_0.46                                                                   ./                                                                              
 added surface contribution
 ./Rayleigh_0.46_F000M030.azi
 added surface contribution
 ./Rayleigh_0.46_F020M030.azi
 added surface contribution
 ./Rayleigh_0.46_F040M030.azi
 added surface contribution
 ./Rayleigh_0.46_F060M030.azi
 added surface contribution
 ./Rayleigh_0.46_F080M030.azi
 added surface contribution
 ./Rayleigh_0.46_F100M030.azi
 added surface contribution
 ./Rayleigh_0.46_F120M030.azi
 added surface contribution
 ./Rayleigh_0.46_F140M030.azi
 added surface contribution
 ./Rayleigh_0.46_F160M030.azi
 added surface contribution
 ./Rayleigh_0.46_F180M030.azi
 added surface contribution
 ./Rayleigh_0.46_F000M035.azi
 added surface contribution
 ./Rayleigh_0.46_F020M035.azi
 added surface contribution
 ./Rayleigh_0.46_F040M035.azi
 add

In [89]:
!ls -t

Rayleigh_0.46.rsp      vec_srf	       tmp		   rdrsp.f90
Rayleigh_0.46	       lib	       vec_interp.f	   cmd_line_gen.f90
benchmark_aerosol.PDA  include	       vec_generate_ops.f  extinction_other.f
Rayleigh_0.46.info     compile_lib.sh  README.txt	   kern_srf.f
fort.21		       atmos_gen       vec_srf.f	   phasem_db.f
Miedb_m80_n19_l0653    IDL	       test		   sphere_db.f
vec_test_case.info     kern_gen        HazeL_Results.txt   testoc.f90
vec_generate_ops       phase_gen       ihm_670_isccp.txt
vec_interp	       surf_gen        vec_generate.f
